In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
test = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')
train = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
sub = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')

In [3]:
train.head()

,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,...,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,...,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,...,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,...,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,...,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [4]:
#train.info()

In [5]:
#train.isna().sum().sum()

In [6]:
def process_data(df):

    df.drop(columns = ['id','Policy Start Date'], inplace = True)

    non_numeric_cols = list(df.select_dtypes(exclude=['number']).columns.values)
    numeric_cols = list(df.select_dtypes(include=['number']).columns.values)
    
    for col in non_numeric_cols:
        df[col] = df[col].fillna(df[col].mode()[0])

    for col in numeric_cols:
        df[col] = df[col].fillna(df[col].mean())
    
    return df

In [7]:
train_data = process_data(train)

In [8]:
#train.isna().sum().sum()

In [9]:
def prepare_data(df):
    ohe = OneHotEncoder(handle_unknown='ignore',sparse_output = False
                       ).set_output(transform = 'pandas')
    scale = StandardScaler()

    numeric_cols = list(df.select_dtypes(include=['number']
                                         ).columns.values)
    df[numeric_cols] = scale.fit_transform(df[numeric_cols])
    
    non_numeric_cols = list(df.select_dtypes(exclude=['number']
                                            ).columns.values)
    for col in non_numeric_cols:
        transform = ohe.fit_transform(df[[col]])
        df.drop(columns = col, inplace = True)
        df = pd.concat([df,transform], axis=1)
    return df

In [10]:
train_data_clean = prepare_data(train_data)

In [11]:
#tst.shape

In [12]:
# Splitting variables
y = train_data_clean[["Premium Amount"]] # dependent
train_data_clean.pop("Premium Amount")
X = train_data_clean


In [13]:
# Splitting dataset into test/train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [14]:
# Regressor model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [15]:
test_data = process_data(test)
test_data_clean = prepare_data(test_data)

In [16]:
pred = regressor.predict(test_data_clean)

In [17]:
sub['Premium Amount'] = pred

In [18]:
sub.to_csv('Submission.csv',index =False)